In [ ]:
import os
import glob
import pathlib
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
DATA_DIR = 'storage/Data/'
DATA_DIR = pathlib.Path(DATA_DIR)
IMG_SIZE = 150
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 20
EPOCHS = 7

In [ ]:
CLASS_NAMES = np.array([item.name for item in DATA_DIR.glob('*')])
CLASS_NAMES

In [ ]:
list_ds = tf.data.Dataset.list_files(glob.glob('storage/Data/*/*'))

In [ ]:
for f in list_ds.take(5):
  print(f.numpy())

In [ ]:
def get_label(file_path):
  parts = tf.strings.split(file_path, os.path.sep)
  if parts[2] == 'Hot Dog':
      return 0
  else:
      return 1

In [ ]:
def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.io.decode_image(img, channels=3, expand_animations=False)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  return tf.image.resize(img, [IMG_SIZE, IMG_SIZE])

In [ ]:
def process_path(file_path):
  label = get_label(file_path)
  # load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label

In [ ]:
labeled_ds = list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

In [ ]:
for image, label in labeled_ds.take(1):
  print("Image shape: ", image.numpy().shape)
  print("Label: ", label.numpy())

In [ ]:
VAL_COUNT = 300
val_label_ds = labeled_ds.take(VAL_COUNT)
train_label_ds = labeled_ds.skip(VAL_COUNT)

In [ ]:
tds = train_label_ds.cache()
tds = tds.shuffle(VAL_COUNT)
tds = tds.repeat()
tds = tds.batch(BATCH_SIZE)
tds = tds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
vds = val_label_ds.shuffle(VAL_COUNT)
vds = vds.repeat()
vds = vds.batch(BATCH_SIZE)
vds = vds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(IMG_SIZE,IMG_SIZE,3)),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(256, (3,3), activation='relu', padding='same'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer='adam', metrics=['binary_accuracy'])

In [ ]:
model.summary()

In [ ]:
steps_per_epoch = 2614 // EPOCHS
steps_per_validation = 300 // EPOCHS

In [ ]:
history = model.fit(tds, epochs=EPOCHS, steps_per_epoch=steps_per_epoch, validation_data=vds, validation_steps=steps_per_validation)

In [ ]:
acc = history.history['binary_accuracy']
val_acc = history.history['val_binary_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
def get_label_name(x):
    if x == 0:
        return 'Hot Dog'
    else:
        return 'Not Hot Dog'

In [ ]:
batch_holder = np.zeros((7, IMG_SIZE, IMG_SIZE, 3))
img_dir='Test/Test/'
for i, img in enumerate(os.listdir(img_dir)):
  img = tf.keras.preprocessing.image.load_img(os.path.join(img_dir,img), target_size=(IMG_SIZE,IMG_SIZE))
  batch_holder[i, :] = img

result=model.predict_classes(batch_holder)
 
fig = plt.figure(figsize=(20, 20))
 
for i,img in enumerate(batch_holder):
  fig.add_subplot(4,5, i+1)
  plt.title(get_label_name(result[i][0]))
  plt.imshow(img/255.)
  
plt.show()

In [ ]:
model.save('hot dog clf_v2.h5', save_format='h5')